<a href="https://colab.research.google.com/github/OdysseusPolymetis/digital_classics_course/blob/main/5_scansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cltk

In [19]:
import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger('CLTK').setLevel(logging.WARNING)

In [2]:
from cltk.prosody.lat.scanner import Scansion

In [ ]:
Scansion().scan_text('dedērunt te miror antōnī quorum. sī quid est in mē ingenī jūdicēs quod sentiō.')

['u--uuu---ux', 'u---u--u---ux']

In [3]:
from cltk.prosody.grc import Scansion

In [ ]:
texte = """
Ἄνδρα μοι ἔννεπε, Μοῦσα, πολύτροπον, ὃς μάλα πολλὰ
πλάγχθη, ἐπεὶ Τροίης ἱερὸν πτολίεθρον ἔπερσε,
πολλῶν δ´ ἀνθρώπων ἴδεν ἄστεα καὶ νόον ἔγνω,
πολλὰ δ´ ὅ γ´ ἐν πόντῳ πάθεν ἄλγεα ὃν κατὰ θυμόν,
ἀρνύμενος ἥν τε ψυχὴν καὶ νόστον ἑταίρων.
ἀλλ´ οὐδ´ ὧς ἑτάρους ἐρρύσατο, ἱέμενός περ.
"""

scanner = Scansion()

verses = texte.strip().split('\n')

resultats_scansion = []

for vers in verses:
    scanned = scanner.scan_text(vers+".")
    resultats_scansion.append(scanned)

for i, resultat in enumerate(resultats_scansion):
    print(f"Vers {i+1}: {resultat}")

INFO:CLTK:IndexError while checking if syllable 'λλα' is long. Continuing.
INFO:CLTK:IndexError while checking if syllable 'ρσε' is long. Continuing.
INFO:CLTK:IndexError while making syllables of 'δ´'. Continuing.
INFO:CLTK:IndexError while checking if syllable 'γνω' is long. Continuing.
INFO:CLTK:IndexError while making syllables of 'δ´'. Continuing.
INFO:CLTK:IndexError while making syllables of 'γ´'. Continuing.
INFO:CLTK:IndexError while checking if syllable 'θυμόν' is long. Continuing.
INFO:CLTK:IndexError while checking if syllable 'ίρων' is long. Continuing.
INFO:CLTK:IndexError while checking if syllable 'περ' is long. Continuing.


Vers 1: ['¯˘¯¯˘˘¯˘˘˘˘¯˘¯x']
Vers 2: ['¯˘¯˘¯˘˘¯˘¯˘˘¯x']
Vers 3: ['¯¯¯¯˘˘¯˘˘¯˘¯x']
Vers 4: ['¯˘˘¯¯˘¯˘˘¯˘˘x']
Vers 5: ['¯˘˘¯¯˘¯¯˘˘˘x']
Vers 6: ['˘˘¯˘˘¯˘˘˘¯x']


In [ ]:
!git clone https://github.com/cltk/grc_text_tesserae.git

Cette partie peut être plus longue, on lui demande de scander tout Homère et de trouver les 5 types de scansion les plus fréquents.

In [20]:
import os
import re
from collections import Counter

scanner = Scansion()

dossier = "/content/grc_text_tesserae/texts"
fichiers = [f for f in os.listdir(dossier) if f.endswith('.tess') and "homer" in f.lower()]

scansions = []

for fichier in fichiers:
    print(f"Traitement du fichier : {fichier}")
    with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
        for ligne in f:
            vers_propre = re.sub(r'<[^>]+>', '', ligne).strip() + '.'

            try:
                scanned_vers = scanner.scan_text(vers_propre)
                scansions.extend(scanned_vers)
            except Exception as e:
                print(f"Erreur de scansion pour le vers : {vers_propre}")
                print(e)

scansion_counter = Counter(scansions)
cinq_plus_frequents = scansion_counter.most_common(5)

for scansion, freq in cinq_plus_frequents:
    print(f"Scansion: {scansion}, Fréquence: {freq}")


Traitement du fichier : homer.odyssey.part.8.tess
Traitement du fichier : homer.odyssey.part.3.tess
Traitement du fichier : homer.iliad.part.5.tess
Traitement du fichier : homer.iliad.part.10.tess
Traitement du fichier : homer.odyssey.part.18.tess
Traitement du fichier : homer.iliad.part.24.tess
Traitement du fichier : homer.iliad.part.15.tess
Traitement du fichier : homer.odyssey.part.14.tess
Traitement du fichier : homer.iliad.part.3.tess
Traitement du fichier : homer.odyssey.part.9.tess
Traitement du fichier : homer.odyssey.part.19.tess
Traitement du fichier : homer.odyssey.part.4.tess
Traitement du fichier : homer.odyssey.part.5.tess
Traitement du fichier : homer.odyssey.part.22.tess
Traitement du fichier : homer.iliad.part.17.tess
Traitement du fichier : homer.iliad.part.8.tess
Traitement du fichier : homer.odyssey.part.24.tess
Traitement du fichier : homer.odyssey.part.23.tess
Traitement du fichier : homer.iliad.part.23.tess
Traitement du fichier : homer.odyssey.part.10.tess
Trai

In [21]:
list_of_sentences=['dedērunt te miror antōnī quorum. sī quid est in mē ingenī jūdicēs quod sentiō.', 'dedērunt te miror antōnī quorum. sī quid est in mē ingenī jūdicēs quod sentiō.']

In [22]:
len(list_of_sentences)

2

In [24]:
from cltk.prosody.lat.scanner import Scansion

In [25]:
for phrase in list_of_sentences:
  result=Scansion().scan_text(phrase)
  print(result)

['u--uuu---ux', 'u---u--u---ux']
['u--uuu---ux', 'u---u--u---ux']


In [ ]:
!git clone https://github.com/cltk/lat_text_latin_library.git

In [ ]:
from cltk.prosody.lat.hexameter_scanner import HexameterScanner

In [ ]:
def is_hexameter(scansion):
    scansion_sans_espaces = re.sub(r'\s+', '', scansion)

    pieds = re.findall(r'(--|-U{0,2})', scansion_sans_espaces)

    dernier_pied_ok = pieds[-1] in ['--', '-U']

    return len(pieds) == 6 and dernier_pied_ok

In [ ]:
def nettoyer_vers(ligne):
    vers_nettoye = re.sub(r'\s*\d+\s*$', '', ligne).strip()
    return vers_nettoye

In [ ]:
from cltk.prosody.lat.hexameter_scanner import HexameterScanner
import os

scanner = HexameterScanner()

dossier = "/content/lat_text_latin_library/vergil"
fichiers = [f for f in os.listdir(dossier) if f.endswith('.txt') and "aen" in f.lower()]

total_vers = 0
for fichier in fichiers:
    with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
        for ligne in f:
            if ligne.strip():
                total_vers += 1

print(f"Nombre total de vers à traiter: {total_vers}")

vers_et_scansions = []
vers_traites = 0

for fichier in fichiers:
    with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
        for ligne in f:
            ligne_propre = nettoyer_vers(ligne)
            if ligne_propre:
                try:
                    scansion = scanner.scan(ligne_propre)
                    scansion_corrige = scanner.correct_invalid_fifth_foot(scansion.scansion)
                    scansion_corrige = scanner.correct_inverted_amphibrachs(scansion_corrige)
                    if is_hexameter(scansion_corrige):
                        vers_et_scansions.append((ligne_propre, scansion_corrige))
                except Exception as e:
                    print(f"Erreur de scansion pour le vers : {ligne_propre}")
                    print(e)

                vers_traites += 1

print(f"Tous les vers ont été traités. Total: {vers_traites}/{total_vers}")

In [ ]:
import random

nombre_vers_a_afficher = min(5, len(vers_et_scansions))

vers_choisis = random.sample(vers_et_scansions, nombre_vers_a_afficher)

for vers, metrique in vers_choisis:
    print(f"Vers: {vers}")
    print(f"Métrique: {metrique}\n")

In [ ]:
from collections import Counter

def normaliser_scansion(scansion):
    return re.sub(r'[^-U]', '', scansion)

hexametres_normalises = [normaliser_scansion(scansion) for vers, scansion in vers_et_scansions]

compteur_hexametres = Counter(hexametres_normalises)

trois_plus_frequents = compteur_hexametres.most_common(3)

for scansion, nombre in trois_plus_frequents:
    print(f"Scansion: {scansion}, Occurrences: {nombre}")